# parameters



In [ ]:
data_dir       = "/media/ansary/DriveData/Work/bengalAI/datasets/imgOCR"
img_height     = 64
img_width      = 512
save_dir       = "/media/ansary/DriveData/Work/bengalAI/datasets/imgOCR"
nb_train       = 1000000
nb_eval        = 1000

# Imports

In [ ]:
#----------------
# images
#----------------
from coreLib.dataset import DataSet
from coreLib.words import single
from coreLib.store import genTFRecords
from coreLib.utils import create_dir

from tqdm.auto import tqdm
import pandas as pd
import numpy as np 
import cv2
import os 
import random
import matplotlib.pyplot as plt
%matplotlib inline

#----------------
# dataset 
#----------------
ds=DataSet(data_dir)
# pairs
img_dir=create_dir(save_dir,"images")
tgt_dir=create_dir(save_dir,"targets")
wrd_dir=create_dir(save_dir,"words")
data_csv=os.path.join(save_dir,"data.csv")
# records
rec_dir  =create_dir(save_dir,"tfrecords")



# Data Creation

In [ ]:
idens=[]
labels=[]
# create the images
for i in tqdm(range(nb_train+nb_eval)):
    try:
        # selection
        comp_type =random.choices(population=["number","grapheme","mixed"],
                                  weights=[0.1,0.8,0.1],
                                  k=1)[0]
        use_dict  =random.choices(population=[True,False],
                                  weights=[0.9,0.1],
                                  k=1)[0]
        img,tgt,word,label=single(ds,
                                  comp_type,
                                  use_dict,
                                  (img_height,img_width))
        # label-mapping
        idens.append(f"{i}.png")
        labels.append(label)


        # save
        cv2.imwrite(os.path.join(img_dir,f"{i}.png"),img)
        cv2.imwrite(os.path.join(tgt_dir,f"{i}.png"),tgt)
        cv2.imwrite(os.path.join(wrd_dir,f"{i}.png"),word)
    except Exception as e:
        print(e)
        
df=pd.DataFrame({"iden":idens,"label":labels})
df.to_csv(data_csv,index=False)
df


In [ ]:
from glob import glob
# paths    
img_paths=[img_path for img_path in glob(os.path.join(img_dir,"*.*"))]
# tfrecords
genTFRecords(img_paths,rec_dir)